In [1]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm

In [2]:
print(tf.__version__)

0.12.0


In [3]:
from data import get_data

In [4]:
X_train, y_train, X_test, y_test = get_data(one_hot=True)

In [5]:
mean_of_train = np.mean(X_train)
std_of_train = np.std(X_train)
print(mean_of_train, std_of_train)

115.481949803 65.2925375088


In [6]:
X_train = (X_train - mean_of_train) / std_of_train
X_test = (X_test - mean_of_train) / std_of_train

In [7]:
train_len, width, height, channel = X_train.shape
test_len = X_test.shape[0]
print("""
训练数据数量：{}
高度:{}
宽度：{}
通道数：{}
测试数据数量：{}
""".format(train_len, width, height, channel, test_len))


训练数据数量：20000
高度:128
宽度：128
通道数：3
测试数据数量：5000



In [8]:
# 载入我的自定义库 @qhduan
import sys
import importlib
sys.path.insert(0, '../../')
import tfkit
importlib.reload(tfkit)

<module 'tfkit' from '../../tfkit/__init__.py'>

In [9]:
# 学习率
learning_rate = 0.01
# 迭代次数（批次）
n_epoch = 50
# 批次大小
# 可能需要调小
batch_size = 32
# dropout
dropout = 0.8
# 输出大小
target_size = y_train.shape[1]
print('target_size is', target_size)

target_size is 2


In [10]:
tf.set_random_seed(0)

In [11]:
# 输入占位符
X = tf.placeholder(tf.float32, [batch_size, width, height, channel])
# 输出占位符
y = tf.placeholder(tf.float32, [batch_size, target_size])

In [12]:
model = X
model = tfkit.conv(model, 64, (5, 5), 'conv_1', strides=(1, 2, 2, 1), activation='relu', batch_normalization=True)
model = tfkit.max_pool(model, 'mp_1')
model = tfkit.dropout(model, dropout)
model = tfkit.conv(model, 128, (5, 5), 'conv_2', strides=(1, 2, 2, 1), activation='relu', batch_normalization=True)
model = tfkit.max_pool(model, 'mp_2')
model = tfkit.dropout(model, dropout)
model = tfkit.conv(model, 256, (3, 3), 'conv_3', strides=(1, 1, 1, 1), activation='relu', batch_normalization=True)
model = tfkit.max_pool(model, 'mp_3')
model = tfkit.dropout(model, dropout)
model = tfkit.flatten(model, 'flatten')
model = tfkit.full_connect(model, 512, name='fc_bn', activation='relu', batch_normalization=True)
model = tfkit.dropout(model, dropout)
model = tfkit.full_connect(model, target_size, 'fc', activation='linear', batch_normalization=True)

===> (32, 62, 62, 64) 	 conv_1
===> (32, 31, 31, 64) 	 mp_1
===> (32, 14, 14, 128) 	 conv_2
===> (32, 7, 7, 128) 	 mp_2
===> (32, 5, 5, 256) 	 conv_3
===> (32, 2, 2, 256) 	 mp_3
===> (32, 1024) 	 flatten
===> (32, 512) 	 fc_bn
===> (32, 2) 	 fc


In [13]:
train_step, cost = tfkit.train_softmax(
    model, y,
    opt=tf.train.AdadeltaOptimizer(learning_rate=learning_rate),
    clip=None
)

In [14]:
measures = [
    cost,
    tfkit.accuracy(model, y, softmax=True),
    tfkit.precision(model, y, softmax=True),
    tfkit.recall(model, y, softmax=True),
    tfkit.f1(model, y, softmax=True)
]

In [15]:
init = tf.global_variables_initializer()

In [16]:
with tf.Session() as sess:
    sess.run(init)
    total = int((len(X_train) - 1) / batch_size + 1)
    for epoch in range(n_epoch):
        print('epoch: {}'.format(epoch))
        record = []
        for batch_x, batch_y in tqdm(tfkit.batch_flow(X_train, y_train, batch_size), total=total, file=sys.stdout):
            sess.run([train_step], feed_dict={X: batch_x, y: batch_y})
            record.append(sess.run(measures, feed_dict={X: batch_x, y: batch_y}))
        print('train: loss: {:.4f}, acc: {:.4f}, prec: {:.4f}, rec: {:.4f}, f1: {:.4f}'.format(
            np.mean([x[0] for x in record]),
            np.mean([x[1] for x in record]),
            np.mean([x[2] for x in record]),
            np.mean([x[3] for x in record]),
            np.mean([x[4] for x in record])
        ))
        record = []
        for batch_x, batch_y in tfkit.batch_flow(X_test, y_test, batch_size):
            record.append(sess.run(measures, feed_dict={X: batch_x, y: batch_y}))
        print('test: loss: {:.4f}, acc: {:.4f}, prec: {:.4f}, rec: {:.4f}, f1: {:.4f}'.format(
            np.mean([x[0] for x in record]),
            np.mean([x[1] for x in record]),
            np.mean([x[2] for x in record]),
            np.mean([x[3] for x in record]),
            np.mean([x[4] for x in record])
        ))

epoch: 0
100%|██████████| 625/625 [00:22<00:00, 28.34it/s]
train: loss: 0.6757, acc: 0.5974, prec: 0.6008, rec: 0.5874, f1: 0.5877
test: loss: 0.6553, acc: 0.6152, prec: 0.6190, rec: 0.6106, f1: 0.6078
epoch: 1
100%|██████████| 625/625 [00:22<00:00, 27.94it/s]
train: loss: 0.6436, acc: 0.6301, prec: 0.6330, rec: 0.6295, f1: 0.6247
test: loss: 0.6457, acc: 0.6276, prec: 0.6308, rec: 0.6305, f1: 0.6229
epoch: 2
100%|██████████| 625/625 [00:22<00:00, 28.00it/s]
train: loss: 0.6324, acc: 0.6429, prec: 0.6443, rec: 0.6445, f1: 0.6379
test: loss: 0.6288, acc: 0.6473, prec: 0.6514, rec: 0.6452, f1: 0.6409
epoch: 3
100%|██████████| 625/625 [00:22<00:00, 28.01it/s]
train: loss: 0.6227, acc: 0.6496, prec: 0.6538, rec: 0.6494, f1: 0.6445
test: loss: 0.6268, acc: 0.6521, prec: 0.6557, rec: 0.6537, f1: 0.6471
epoch: 4
100%|██████████| 625/625 [00:22<00:00, 27.83it/s]
train: loss: 0.6112, acc: 0.6655, prec: 0.6694, rec: 0.6662, f1: 0.6608
test: loss: 0.6178, acc: 0.6576, prec: 0.6637, rec: 0.6529, f